Preparing a dataset for Basenji training involves a series of design choices.

The input you bring to the pipeline is:
* BigWig coverage tracks
* Genome FASTA file

First, make sure you have an hg19 FASTA file visible. If you have it already, put a symbolic link into the data directory. Otherwise, I have a machine learning friendly simplified version you can download in the next cell.

In [9]:
import os, subprocess

if not os.path.isfile('data/hg19.ml.fa'):
    subprocess.call('curl -o data/hg19.ml.fa https://storage.googleapis.com/basenji_tutorial_data/hg19.ml.fa', shell=True)
    subprocess.call('curl -o data/hg19.ml.fa.fai https://storage.googleapis.com/basenji_tutorial_data/hg19.ml.fa.fai', shell=True)                

Next, let's grab a few CAGE datasets from FANTOM5 related to heart biology.

These data were processed by
1. Aligning with Bowtie2 with very sensitive alignment parameters.
2. Distributing multi-mapping reads and estimating genomic coverage with [bam_cov.py](https://github.com/calico/basenji/blob/master/bin/bam_cov.py)

In [ ]:
if not os.path.isfile('data/CNhs11760.bw'):
    subprocess.call('curl -o data/CNhs11760.bw https://storage.googleapis.com/basenji_tutorial_data/CNhs11760.bw', shell=True)
    subprocess.call('curl -o data/CNhs12843.bw https://storage.googleapis.com/basenji_tutorial_data/CNhs12843.bw', shell=True)
    subprocess.call('curl -o data/CNhs12856.bw https://storage.googleapis.com/basenji_tutorial_data/CNhs12856.bw', shell=True)_

Then we'll write out these BigWig files and labels to a samples table.

In [ ]:
samples_out = open('data/heart_wigs.txt', 'w')
print('aorta\tdata/CNhs11760.bw', file=samples_out)
print('artery\tdata/CNhs12843.bw', file=samples_out)
print('pulmonic_valve\tdata/CNhs12856.bw', file=samples_out)
samples_out.close()

Next, we want to choose genomic sequences to form batches for stochastic gradient descent, divide them into training/validation/test sets, and form a single file to provide to downstream programs.

The script [basenji_hdf5_single.py](https://github.com/calico/basenji/blob/master/bin/basenji_hdf5_single.py) implements this procedure.

The most relevant options here are:

| Option/Argument | Value | Note |
|:---|:---|:---|
| -d | 0.1 | Down-sample the genome to 10% to speed things up here. |
| -g | data/unmap_macro.bed | Dodge large-scale unmappable regions like assembly gaps. |
| -l | 131072 | Sequence length. |
| -o | data/heart_l131k.bed | Write out the chosen sequences to a BED file. |
| -p | 4 | Uses multiple threads with the multiprocessing library. |
| -s | 65536 | Stride the sequences. By setting this to half the sequence length, the sequences will overlap by half. 
| -t | chr9 | Hold out chr9 sequences for testing. |
| -w | 128 | Pools the nucleotide-resolution values to 128 bp bins. |
| -v | chr8 | Hold out chr8 sequences for validation. |
| fasta_file| data/hg19.ml.fa | FASTA file to extract sequences from. |
| sample_wigs_file | data/heart_wigs.txt | Samples and BigWig paths. |
| hdf5_file | data/heart_l262k.h5 | Output HDF5 file. |

In [7]:
! basenji_hdf5_single.py -d .2 -g data/unmap_macro.bed -l 131072 -o data/heart_l131k.bed -p 4 -s 65536 -t chr9 -w 128 -v chr8 data/hg19.ml.fa data/heart_wigs.txt data/heart_l131k.h5

/Users/davidkelley/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/davidkelley/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
7022 sequences one hot coded
Reading and pre-processing bigwigs for 738 segments
2018-05-14 12:49:56.831948: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.2 AVX AVX2 FMA
Tiling from chr11:76680786-77037939
Tiling from chrX:37259843-37420898
Tiling from chr8:69280152-69656090
Tiling from chr15:62297852-64618849
Tiling from chr11:23325708-24349490
Tili

Now, you can offer data/heart_l131k.h5 to [basenji_train.py](https://github.com/calico/basenji/blob/master/bin/basenji_train.py) to train a model.

Inside are the following data structures.

In [9]:
import h5py

ml_h5 = h5py.File('data/heart_l131k.h5')
print(list(ml_h5.keys()))

['pool_width', 'target_ids', 'target_labels', 'target_strands', 'test_in', 'test_out', 'test_out_full', 'train_in', 'train_out', 'valid_in', 'valid_out']


In [10]:
print('train_in', ml_h5['train_in'].shape)
print('train_out', ml_h5['train_out'].shape)

print('valid_in', ml_h5['valid_in'].shape)
print('valid_out', ml_h5['valid_out'].shape)

print('test_in', ml_h5['test_in'].shape)
print('test_out', ml_h5['test_out'].shape)

train_in (6240, 131072, 4)
train_out (6240, 1024, 3)
valid_in (338, 131072, 4)
valid_out (338, 1024, 3)
test_in (444, 131072, 4)
test_out (444, 1024, 3)
